<a href="https://colab.research.google.com/github/sherlockieee/cs156/blob/main/lba/CS156_LBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thanks to Finn for helping me with the code, and Khanh for our coworking sessions.